# Comparison of Hessian-vector product

Often times we need Hessian-vector products in numerical optimization. A nice family of examples of using Hessian-vector products is Krylov subspace based iterative algorithms such as Conjugate Gradient, Lanczos method, and Power method. In deep learning literature, Krylov subspace is often used in the context of 2nd-order optimization. Please refer to [Deep learning via Hessian-free optimization](http://www.cs.toronto.edu/~jmartens/docs/Deep_HessianFree.pdf), [Krylov Subspace Descent for Deep Learning](http://www1.icsi.berkeley.edu/~vinyals/Files/vinyals_aistats12.pdf), and [Identifying and attacking the saddle point problem in
high-dimensional non-convex optimization](https://arxiv.org/pdf/1406.2572v1.pdf) for more detail. 

There are three ways to compute Hessian-vector products that we can try in TensorFlow.

1. Compute Hessian directly and multiply the result with a vector.
2. Use finite differences to get Hessian and multiply the result with a vector. (used in Marten's paper)
3. Use Pearlmutter trick. (will explain later)

In this post, I will compare these methods in terms of numerical stability and computational speed. 
I'm mainly interested in how well the finite differences would work when the parameter space gets big / when the condition number of Hessian becomes large, compared to Pearlmutter trick. 

The Method 1 is just for illustrative purpose. In practice, one should not compute Hessian directly because it's too much computation. 

The reason why I included method 2 is just for reference. I've tried to compute Hessian-vector product once in Torch7, which doesn't have an easy way to do the method 3. For TensorFlow / Theano users, Method 3 should suffice.   

I made a TensorFlow function to compute Hessians for multilayer perceptron before, so I'll reuse it to try out the method 1.

In [5]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import math
import numpy as np
import tensorflow as tf
import time

#from datasets import dataset_utils

# Main slim library
slim = tf.contrib.slim

# Speed comparison

## Method 1

In [ ]:
def method1(v):
    H_1 = getHessian()# Did I make this function before?
    return np.dot(H_1, v)

## Method 2

In [23]:
# keras experiments: how graph is created
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Activation
def testKeras(g):
    with g.as_default():
        with tf.Session() as sess:
            K.set_session(sess)
            
            # this placeholder will contain our input digits, as flat vectors
            # img = tf.placeholder(tf.float32, shape=(None, 784))
            
            print('success')
            # from keras import backend as K
            # from keras.models import Sequential
            # from keras.layers import Dense, Activation
            model = Sequential()
            input_size = 64
            # c = tf.constant(4.0)
            input_ph = K.placeholder(shape=(None, input_size))
            model.add(Dense(output_dim=25, input_dim=input_size))
            model.add(Activation("sigmoid"))
            model.add(Dense(output_dim=64, input_dim=25))
            model.add(Activation("sigmoid")) 
            
            # assert model.graph is g
            # print(model.summary())
            g2 = tf.get_default_graph()
            assert g is g2
            

In [24]:
testKeras()

success


In [ ]:
def method2(g, v):
    with g.as_default():
        
        

In [ ]:
def method2(v):
    with tf.Graph().as_default():
        inputs = 
        targets = 
        # Compute finite differences of gradients
        output, end_points = build_MLP(inputs, n_hidden=10)
        # loss = slim.losses.sum_of_squares(predictions, targets) # Slim way
        # The total loss is the uers's loss plus any regularization losses.
        # total_loss = slim.losses.get_total_loss() # Slim way
        loss = tf.reduce_mean(tf.square(output - targets))
        grad = tf.gradients(loss) # make sure that this returns a vector, not a list of gradient and something else.
        loss_eps = tf.add(loss, eps)
        grad_eps = tf.gradients(loss_eps) # make sure that this returns a vector, not a list of gradient and something else.
        Hv = tf.div(grad - grad_eps, eps)
        with tf.Session() as sess:
            sess.run(tf.initialize_all_variables())
            print(sess.run(Hv))

In [6]:
def build_MLP(inputs, n_hidden=5):
    end_points = {}
    with slim.arg_scope([slim.fully_connected], 
                        activation_fn=tf.nn.relu,
                        weights_regularizer=slim.l2_regularizer(0.01)):
        net = slim.fully_connected(inputs, n_hidden, scope="fc1")
        end_points["fc1"] = net 
        output = slim.fully_connected(net, n_output, activation_fn=None, scope="output")
        end_points["output"] = output
        
        return output, end_points

## Method 3 : Pearlmutter trick

Pearlmutter trick is a way to compute $Hv$ in an efficient way using backpropagation. The basic idea is to take the gradient of ----
$$
\nabla_{\theta}(g^T v) = H^T v  = Hv 
$$
, where g is a gradient of the objective funtion with respect to parameters. 

In [8]:
def method3(v):
    with tf.Graph().as_default():
        inputs = 
        targets = 
        output, end_points = build_MLP(inputs, n_hidden=10)
        loss = tf.reduce_mean(tf.square(output - targets))
        grad = tf.gradients(loss) # make sure that this returns a vector, not a list of gradient and something else.
        Hv = tf.gradients(tf.mul(grad, v))
        with tf.Session() as sess:
            sess.run(tf.initialize_all_variables())
            print(sess.run(Hv))

SyntaxError: invalid syntax (<ipython-input-8-5bc8f0f6582b>, line 3)

# Condition number 

Condition number captures sensitivity of a system, which has inputs and outputs. It measures how outputs will change with respect to change in the inputs. Many problems **(examples?)** can be reduced to finding $x$ s.t. $Ax = b$. In this case, $b$ is the input and $x$ is the solution we want, and $A$ represents the structure of the problem or system you are interested in. There are a number of types of condition number depending on how to measure the "sensitivity" of the system. Let us focus on the case where $A$ is a symmetric matrix because then the definition of condition number is simple: 

---

Condition number of a symmetric matrix $A$ is defined as 
$$ 
\frac{|\lambda_{max}|}{|\lambda_{min}|} 
$$

---

where $\lambda_{max / min}$ is the maximum / minimum eigenvalue of the matrix $A$. For those who are interested to know why it is so, please refer to my other blog post on [induced matrix norm](). 

Let us use a simple quadratic function for our objective funtion because it's easier to control the condition number of the Hessian; we can control the condition number of a matrix through SVD by modifying the resulting eigenvalues of the diagonal matrix returned by SVD.  

For any quadratic function s.t. $f(x) = \frac{1}{2} x^T A x + b^T x + c$, the Hessian of $f(x)$ is $A$ when $A$ is symmetric. So we first create a random symmetric matrix like this:

In [7]:
def createRandomSymmMatrix(n_dim, condition_num): 
    random_mat = np.random.random(n_dim*n_dim).reshape(n_dim, n_dim) 
    random_mat = np.dot(np.transpose(random_mat), random_mat) # For any X, X^T X is always symmetric. 
    U,S,Vt = np.linalg.svd(random_mat) # Do svd to get diagonal elements. Note that S is a vector. 
    S = np.repeat(1.0, n_dim) # Replace the diagonal elements with a vector of 1s. 
    S[-1] = 1.0/condition_num # Let the last element of S be very small. This will determine condition number. 
    return np.dot(np.dot(U, np.diag(S)), Vt)

In [8]:
createRandomSymmMatrix(2, 2) 

array([[ 0.83938984,  0.2334726 ],
       [ 0.2334726 ,  0.66061016]])

# Stability comparison

Now, we will compare Method2 and Method3 in terms of the stability. 

## Method 2: Finite differences  

To compute $Hv$ using finite differences, we use the following identity:

$$
Hv = \lim_{\epsilon \rightarrow 0} \frac{\nabla f(\theta - \epsilon v) - \nabla f(\theta + \epsilon v)}{2\epsilon}
$$

In [42]:
def method2_c(v, inputs, eps):
    g2 = tf.Graph()
    with g2.as_default():
        # how to perturb weights
        with g2.name_scope("grad1") as scope:
            output1 = build_quadratic(inputs, eps=-eps, v=v)  
            loss1 = tf.reduce_sum(output1 - 0)
            tvars1 = tf.get_collection(tf.GraphKeys.VARIABLES, scope=scope) 
            for i in tvars1: print(i.name)
            grad1 = tf.gradients(loss1, tvars1)[0]
        with g2.name_scope("grad2") as scope:
            output2 = build_quadratic(inputs, eps=eps, v=v)  
            loss2 = tf.reduce_sum(output2 - 0) 
            tvars2 = tf.get_collection(tf.GraphKeys.VARIABLES, scope=scope)
            for i in tvars2: print(i.name)
            grad2 = tf.gradients(loss2, tvars2)[0]
        Hv = 0.5*(grad1-grad2) / eps 
        with tf.Session() as sess:
            sess.run(tf.initialize_all_variables())
            return sess.run(Hv)

In [62]:
np.linalg.cond(inputs['A']) 

199.99999999999687

In [65]:
n_dim = 2000
condition_num = 200
vec = np.repeat(1.0, n_dim).reshape([n_dim, 1])
inputs = createInputs(n_dim, condition_num) 
eps = 1e-5 # what value should I use 
Hv_method2 = method2_c(vec, inputs, eps) 
trueHv = np.dot(np.transpose(inputs['A']), vec) # true value
print(np.linalg.norm(Hv_method2 - trueHv))
Hv_method3 = method3_c(vec, inputs, n_dim, condition_num)
print(np.linalg.norm(Hv_method3 - trueHv)) 

grad1/Variable:0
grad1/Variable_1:0
grad2/Variable:0
grad2/Variable_1:0
89.4427190907
5.51851954553e-11


In [ ]:
Hv_method3 = method3_c(vec, inputs, n_dim, n_cond_num) 
print(np.dot(np.transpose(inputs['A']), vec)) # true value 

## Method 3: Pearlmutter Trick

In [2]:
def createInputs(n_dim, condition_num):
    inputs = dict() 
    inputs["A"] = createRandomSymmMatrix(n_dim, condition_num)
    inputs["b"] = np.random.random(n_dim).reshape(n_dim, 1)
    inputs["c"] = np.random.random(1)
    return inputs 

In [53]:
def method3_c(v, inputs, n_dim, condition_num):
    with tf.Graph().as_default():
        output = build_quadratic(inputs, eps=0, v=v)  
        loss = tf.reduce_sum(output - 0)
        tvars = tf.trainable_variables()
        grad = tf.gradients(loss, tvars)[0]
        # Pearlmutter Trick
        Hv = tf.gradients(tf.matmul(tf.transpose(grad), v), tvars)
        with tf.Session() as sess:
            sess.run(tf.initialize_all_variables())
            #print(sess.run(grad).shape)
            #print(sess.run(tf.matmul(tf.transpose(grad), v)))
            #print(sess.run(Hv))
            return sess.run(Hv)

In [20]:
n_dim = 5
condition_num = 2
vec = np.repeat(1.0, n_dim).reshape([n_dim, 1])
inputs = createInputs(n_dim, condition_num)
Hv_method3 = method3_c(vec, inputs, n_dim, n_cond_num) 
print(np.dot(np.transpose(inputs['A']), vec)) # true value

[[ 0.87837996 -0.15390476  0.04465794  0.14183629 -0.01483392]
 [-0.15390476  0.80524034  0.05651264  0.17948753 -0.01877167]
 [ 0.04465794  0.05651264  0.98360195 -0.05208119  0.0054469 ]
 [ 0.14183629  0.17948753 -0.05208119  0.83458703  0.01729968]
 [-0.01483392 -0.01877167  0.0054469   0.01729968  0.99819072]]
(5, 1)
[[ 8.28377084]]
[array([[ 0.8961355 ],
       [ 0.86856408],
       [ 1.03813824],
       [ 1.12112934],
       [ 0.98733171]]), array([[ 0.8961355 ],
       [ 0.86856408],
       [ 1.03813824],
       [ 1.12112934],
       [ 0.98733171]])]
[[ 0.8961355 ]
 [ 0.86856408]
 [ 1.03813824]
 [ 1.12112934]
 [ 0.98733171]]


In [23]:
def build_quadratic(inputs, eps, v):
    A = inputs["A"]
    # print(A)
    b = inputs["b"]
    c = inputs["c"]
    n_dim, _ = np.shape(A)
    
    x_ = tf.Variable(np.float64(np.repeat(1.0,n_dim).reshape(n_dim,1)))
    assert v.shape == (n_dim, 1)
    epsv = tf.Variable(np.float64(eps*v))
    x = x_ + epsv
    
    # Construct the computational graph for quadratic function: f(x) = 1/2 * x^t A x + b^t x + c
    output = 0.5 * tf.matmul(tf.matmul(tf.transpose(x), A), x) + tf.matmul(tf.transpose(b), x) + c
    return output

In [41]:
np.linalg.cond(createMatrix(3,10)) 

10.000000000000005

# Comparison

In [ ]:
n_dim = 20
condition_num_list = 
vec = np.repeat(1.0, n_dim).reshape([n_dim, 1])
# inputs = createInputs(n_dim, condition_num)
eps = 1e-5 # what value should I use 
Hv_method3 = method2_c(vec, inputs, eps) 
print(np.dot(np.transpose(inputs['A']), vec)) # true value

Looks good. 

## Reference

Next, let's compare the result of the above function with a method using finite differences. 

(By the way, there is a way to compute Hessian-vector product using only gradients, which is called Pearlmutter trick in neural network literature. Please refer to https://cswhjiang.github.io/2015/10/13/Roperator/ for more details.)